<a href="https://colab.research.google.com/github/TYH71/AIML_CA1/blob/main/Model_Selection_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PAI CA2 - Model Selection

- Class: DAAA/FT/2B/04
- Names: Yu Hoe, Dylan, Choon Wei
- Author: Yu Hoe, Dylan

## Objective

> We aim to conduct a data science project workflow on **Just Taxi** telematic dataset, which includes applying advanced data processing techniques on the dataset, building and optimizing machine learning models, tracking and monitoring machine learning, and deploy the machine-learning model as a graphical user interface.

## Motivation

> This notebook is created to conduct the model selection process, to determine the most suitable machine learning algorithm for the **Just Taxi** telematic dataset.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
! pip install -q --upgrade imbalanced-learn
import imblearn

# Machine Learning Modules
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Matplotlib styling
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = 16, 9
plt.rcParams['figure.autolayout'] = True

## Hardware Specifications

|![Colab Logo](https://colab.research.google.com/img/colab_favicon_256px.png)|
|:--:|
|Google Colaboratory Logo [[Source](https://blog.tensorflow.org/2018/05/colab-easy-way-to-learn-and-use-tensorflow.html)]|

Having the right resources is important to any data science project, which provides an increase in performance and quality. In our case, every team member has purchased [Colab Pro](https://colab.research.google.com/signup) for better GPUs and memory, in order to avoid any bottlenecks in computational performances.



In [3]:
# Retrieve CPU and GPU information
! lscpu
! nvidia-smi

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

# Mounting Google Drive

![Colab + Drive](https://miro.medium.com/max/956/1*BKyVeBWvz1TjhjkBLHpEsg.png)

> In the previous phase of the project, our team has found much difficulty in collaborating on [GitHub](https://github.com/), as the `.csv` dataset crossed over the 100mb git limit. This brought the Waterflow workflow problem, where only one person could attempt their task while the rest stay idle. <br><br> For this phase of the project, we diverted away from this approach and used Google Drive instead to host the dataset such that every team member can have easy access to the dataset via [Google Colaboratory](https://colab.research.google.com/) during development process.

```python
# With only two lines of code, we can access Google Drive.
from google.colab import drive
drive.mount('/content/drive')
```


In [4]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Metadata

The telematics data collected during driving trips are useful to detect if the trip is dangerous and has high crash / accident probability.

Data Schema (Field & Description):

| bookingID | trip |
| ---:|:--- |
| accuracy | accuracy inferred by GPS in meters |
| bearing | GPS bearing in degree |
| acceleration_x | accelerometer reading at x axis (m/s2) |
| acceleration_y | accelerometer reading at y axis (m/s2) |
| acceleration_z | accelerometer reading at z axis (m/s2) (Acceleration determines the acceleration / vibration of the device in motion. Each of the axis can be thought of as a different sensor even though they reside on the same physical chip) |
| gyro_x | gyroscope reading in x axis (rad/s) |
| gyro_y | gyroscope reading in y axis (rad/s) |
| gyro_z | gyroscope reading in z axis (rad/s) (Gyroscope data determine the orientation of the device relative to earth's gravity) |
| second | time of the record by number of seconds |
| speed | speed measured by GPS in m/s |
| label | tags to indicate dangerous driving trip (0: Normal trip / 1: Dangerous trip) |

!! IMPORTANT NOTE: ACCURACY MEANS THE DISTANCE BETWEEN THE PHONE LOCATION AND WHERE IT IS ACTAULLY LOCATED !!

In [5]:
# Import dataset
df = pd.read_csv('/content/drive/MyDrive/PAI Documents/Processed Data/processed_telematic_data.csv')
display(df)

,BookingID,Second,Accuracy,Bearing,Acceleration_x,Acceleration_y,Acceleration_z,Gyro_x,Gyro_y,Gyro_z,Speed,Label
0,0,2.0,8.0,143.29830,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,0.228454,0
1,0,3.0,8.0,143.29830,-1.416705,-9.548032,-1.860977,-0.022413,0.005049,-0.025753,0.228454,0
2,0,9.0,8.0,143.29830,-0.346924,-9.532629,-1.204663,0.014962,-0.050033,0.025118,0.228454,0
3,0,11.0,8.0,143.29830,-0.600986,-9.452029,-2.157507,0.004548,-0.011713,-0.004078,0.228454,0
4,0,12.0,8.0,143.29830,-0.597546,-9.863403,-1.672711,-0.000401,0.000315,-0.009830,0.228454,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7018380,1709396983975,552.0,6.0,185.44942,-0.654221,-8.646927,-4.137964,0.017065,-0.127480,-0.079793,1.695058,1
7018381,1709396983975,553.0,6.0,194.25633,-0.260342,-8.792276,-4.570722,-0.018650,-0.115063,-0.046133,1.695058,1
7018382,1709396983975,554.0,7.0,198.52795,0.179144,-8.934634,-4.572067,-0.015760,-0.018210,-0.006609,1.035811,1
7018383,1709396983975,555.0,8.0,198.52795,-0.585284,-9.059647,-3.930856,-0.013872,0.009509,-0.008706,1.035811,1


In [6]:
# aggregate by bookingid
df_agg = pd.DataFrame()
for col in df.columns:
    if col != "BookingID" and col != "Label":
        temp = df.groupby("BookingID")[col].agg(["mean", "max", "min"])
        df_agg[col + "_mean"] = temp["mean"]
        df_agg[col + "_max"] = temp["max"]
        df_agg[col + "_min"] = temp["min"]
    elif col == "Label":
        temp = df.groupby("BookingID")[col].agg(["max"])
        df_agg[col] = temp['max']

# display dataframe
df_agg.reset_index(inplace=True)
display(df_agg)

# display dataframe shape
print("Before Aggregation: {}".format(df.shape))
print("After Aggregation: {}".format(df_agg.shape))
print("Rows Pct Change: {:.2%}".format((df.shape[0] - df_agg.shape[0])/df.shape[0]))

,BookingID,Second_mean,Second_max,Second_min,Accuracy_mean,Accuracy_max,Accuracy_min,Bearing_mean,Bearing_max,Bearing_min,Acceleration_x_mean,Acceleration_x_max,Acceleration_x_min,Acceleration_y_mean,Acceleration_y_max,Acceleration_y_min,Acceleration_z_mean,Acceleration_z_max,Acceleration_z_min,Gyro_x_mean,Gyro_x_max,Gyro_x_min,Gyro_y_mean,Gyro_y_max,Gyro_y_min,Gyro_z_mean,Gyro_z_max,Gyro_z_min,Speed_mean,Speed_max,Speed_min,Label
0,0,928.337054,1589.0,2.0,9.787946,16.000,4.000,176.962977,359.97977,0.037464,-0.706297,4.782614,-4.692294,-9.606546,-6.119916,-12.190784,-1.534789,2.318857,-6.251807,0.002182,0.197898,-0.392537,-0.003596,0.428893,-0.516878,-0.001543,0.338149,-0.210122,9.127778,22.882523,0.064309,0
1,1,575.399485,1034.0,0.0,3.703376,6.000,3.000,127.372423,337.00000,25.000000,-0.503200,3.387767,-5.352994,9.533105,11.435962,6.623425,-2.202569,1.009146,-5.145321,-0.001151,0.126536,-0.100130,-0.004788,0.470837,-0.600795,0.002577,0.136467,-0.119799,7.729992,21.882141,0.000000,1
2,2,320.777778,822.0,10.0,3.821759,8.000,3.000,179.475309,330.00000,11.000000,0.287461,1.956122,-1.565856,9.735713,10.901134,8.248277,0.069421,1.755003,-2.825244,-0.001514,0.282162,-0.155218,-0.013358,0.344953,-0.418629,0.004042,0.112942,-0.081313,2.851901,9.257438,0.000000,1
3,4,541.207039,1092.0,1.0,10.000000,10.000,10.000,150.385762,351.40985,2.271227,-0.380176,2.019635,-2.866458,-9.371676,-7.135416,-12.916333,-2.654539,0.296381,-5.029947,-0.023892,0.148668,-0.230107,0.023619,0.652628,-0.389871,0.002339,0.505220,-0.209906,6.048292,19.560000,0.000000,1
4,6,540.685437,1092.0,1.0,4.631772,12.000,3.000,195.625243,346.00000,0.000000,0.565690,5.008823,-2.013585,9.504979,13.249917,0.594051,2.314074,6.263423,-1.438959,0.006719,0.362202,-0.200405,-0.009303,0.434384,-0.621802,0.001852,0.258855,-0.336128,4.714045,16.394695,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19927,1709396983957,533.340090,1044.0,0.0,3.549740,14.601,3.000,174.596847,359.00000,0.000000,0.103698,11.296188,-15.993755,1.827002,39.845520,-34.944440,9.778801,42.316414,-24.088799,0.027901,2.139516,-0.866552,-0.013217,0.935871,-1.425123,-0.012467,1.884930,-2.603097,2.483309,6.275580,0.000290,1
19928,1709396983960,424.188623,809.0,2.0,11.808383,16.000,4.000,178.551648,350.57666,10.027151,0.560114,3.709992,-3.379367,-8.704715,-6.629535,-11.460449,-4.471745,-1.930212,-7.562341,-0.012863,0.290076,-0.641901,0.006026,0.477245,-0.376187,-0.004241,0.233631,-0.496440,7.593701,24.059150,0.000000,1
19929,1709396983966,584.032258,988.0,1.0,9.379500,13.936,5.953,197.983871,359.00000,0.000000,-0.201059,2.232147,-2.704697,9.062923,11.092163,7.785767,3.336205,5.963333,0.272308,0.001111,0.122772,-0.172302,-0.005624,0.392319,-0.683594,-0.001112,0.151810,-0.252991,12.739501,25.640000,0.000000,1
19930,1709396983971,523.682773,1076.0,1.0,4.773118,16.000,3.000,205.259454,357.00000,1.000000,0.733790,4.637710,-3.550709,8.901846,11.430268,5.542746,3.772809,9.347249,0.993624,0.001897,0.652905,-0.696496,0.001688,3.940942,-1.047383,-0.000023,0.929654,-0.521151,6.001161,19.287226,0.000000,1


Before Aggregation: (7018385, 12)
After Aggregation: (19932, 32)
Rows Pct Change: 99.72%


In [7]:
# Define Features and Labels
X = df_agg.set_index('BookingID').drop('Label', axis=1)
y = df_agg.set_index('BookingID').loc[:, 'Label']

print("X shape: {}\ny shape: {}".format(X.shape, y.shape))

X shape: (19932, 30)
y shape: (19932,)


In [8]:
# Split Training and Testing Ratio - 4:1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train: {}\ty_train: {}\nX_test: {}\ty_test: {}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

X_train: (15945, 30)	y_train: (15945,)
X_test: (3987, 30)	y_test: (3987,)


# Pipeline Development

In [9]:
# pipeline and preprocessing modules
from imblearn.pipeline import Pipeline
from sklearn.neighbors import LocalOutlierFactor
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

## Local Outlier Factor

In [10]:
# Identify outliers via LocalOutlierFactor
iso = LocalOutlierFactor(contamination=0.05)
X_out = iso.fit_predict(X_train)
# select all rows that are not outliers
mask = (X_out != -1)

# Store removed outliers in a new variable
X_train_out, y_train_out = X_train[mask], y_train[mask]

# summarize the shape of the updated training dataset
print('Shape of the training dataset after removing outliers', X_train_out.shape, y_train_out.shape)

Shape of the training dataset after removing outliers (15147, 30) (15147,)


## SMOTE

In [11]:
# instantiate smote class and resample
smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=5, n_jobs=-1)
X_train_smote, y_train_smote = smote.fit_resample(X_train_out, y_train_out)

print("Before Resampling: {} {}".format(X_train_out.shape, y_train_out.shape))
print("After Resampling: {} {}".format(X_train_smote.shape, y_train_smote.shape))

Before Resampling: (15147, 30) (15147,)
After Resampling: (22904, 30) (22904,)


## Z-Score Standardization

In [12]:
# instantiate standard scaler class
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train_smote, y_train_smote)
X_train_scale = pd.DataFrame(X_train_scale, columns=X_train_smote.columns)

## Create Pipeline Function

In [13]:
# function to build a machine learning pipeline
def create_pipeline(model):
    # model: expects an estimator object, being able to .fit() and .predict()
    # returns a pipeline instance
    ml_pipeline = Pipeline(
        steps=[
            ("SMOTE", smote),
            ("Z-Score Standardization", scaler),
            (model.__repr__(), model)
        ]
    )
    return ml_pipeline

# attempting to recreate ml baseline
ml_baseline = create_pipeline(LogisticRegression(max_iter=1000))
print(ml_baseline)

Pipeline(steps=[('SMOTE', SMOTE(n_jobs=-1, random_state=42)),
                ('Z-Score Standardization', StandardScaler()),
                ('LogisticRegression(max_iter=1000)',
                 LogisticRegression(max_iter=1000))])


# Model Selection

In [14]:
# scikit-learn machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    GradientBoostingClassifier, 
    RandomForestClassifier,
    AdaBoostClassifier,
)

# third party machine learning models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# model selection modules
from sklearn.model_selection import cross_validate

In [15]:
# prepare ray parallel backend
import joblib
!pip install -q ray
from ray.util.joblib import register_ray
import ray
ray.init(ignore_reinit_error=True)
register_ray()

In [16]:
models = [
    LogisticRegression(n_jobs=-1),
    GaussianNB(),
    KNeighborsClassifier(),
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_jobs=-1),
    XGBClassifier(n_jobs=-1),
    LGBMClassifier(n_jobs=-1)
]

In [17]:
# # setting model at default hyperparameters into a list
# models = [
#     LogisticRegression(n_jobs=-1),
#     KNeighborsClassifier(),
#     LinearSVC(),
#     SVC(),
#     DecisionTreeClassifier(),
#     GradientBoostingClassifier(),
#     RandomForestClassifier(n_jobs=-1),
#     AdaBoostClassifier(),
# ]


In [18]:
%%time
models_cv_score = {}

# model training using ray backend
with joblib.parallel_backend('ray'):
    for model in models:
        print(model.__repr__())
        pipeline_obj = create_pipeline(model)
        cv_score = cross_validate(
            pipeline_obj, 
            X_train_out, 
            y_train_out, 
            return_train_score=True,
            cv=10,
            n_jobs=-1, 
            scoring=['accuracy', 'f1', 'precision', 'recall', 'roc_auc'],
            verbose=1
        )
        cv_score_mean = {"mean_{}".format(k) : np.mean(v) for k, v in cv_score.items()}

        models_cv_score[model.__repr__()] = cv_score_mean

2022-02-06 09:21:12,056	WARNING pool.py:494 -- The 'context' argument is not supported using ray. Please refer to the documentation for how to control ray initialization.


LogisticRegression(n_jobs=-1)


[Parallel(n_jobs=-1)]: Using backend RayBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.1s finished


GaussianNB()


[Parallel(n_jobs=-1)]: Using backend RayBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.9s finished


...

[Parallel(n_jobs=-1)]: Using backend RayBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   28.4s finished


LGBMClassifier()


[Parallel(n_jobs=-1)]: Using backend RayBackend with 4 concurrent workers.


CPU times: user 24.6 s, sys: 4.29 s, total: 28.9 s
Wall time: 11min 4s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.6s finished


![](https://www.researchgate.net/profile/Sebastian-Bittrich/publication/330174519/figure/fig1/AS:711883078258689@1546737560677/Confusion-matrix-Exemplified-CM-with-the-formulas-of-precision-PR-recall-RE.png)

In [23]:
models_cv_df = pd.DataFrame(models_cv_score).T
models_cv_df_style = models_cv_df.style.apply(lambda x: ['background-color: yellow; color: black' if v else '' for v in x == x.max()]).apply(lambda x: ['background-color: orange; color: black' if v else '' for v in x == x.min()])
display(models_cv_df_style)

,mean_fit_time,mean_score_time,mean_test_accuracy,mean_train_accuracy,mean_test_f1,mean_train_f1,mean_test_precision,mean_train_precision,mean_test_recall,mean_train_recall,mean_test_roc_auc,mean_train_roc_auc
LogisticRegression(n_jobs=-1),0.739155,0.016379,0.649372,0.653353,0.465099,0.470368,0.370612,0.374926,0.624628,0.631003,0.707766,0.712220
GaussianNB(),0.829836,0.027878,0.681661,0.679474,0.433172,0.429698,0.395317,0.391780,0.500089,0.497647,0.670550,0.671491
KNeighborsClassifier(),0.751049,5.218845,0.582624,0.753072,0.408378,0.638189,0.312137,0.496611,0.590526,0.892708,0.622854,0.898071
SVC(),43.699216,6.156550,0.676836,0.725725,0.434082,0.522247,0.379111,0.454071,0.508247,0.614524,0.678092,0.761598
DecisionTreeClassifier(),1.615918,0.015562,0.652407,1.000000,0.360973,1.000000,0.327325,1.000000,0.402720,1.000000,0.567847,1.000000
RandomForestClassifier(n_jobs=-1),13.703251,0.304821,0.745163,1.000000,0.391103,1.000000,0.470573,1.000000,0.335594,1.000000,0.690340,1.000000
XGBClassifier(n_jobs=-1),6.975440,0.033663,0.743514,0.759182,0.402890,0.439589,0.467566,0.509669,0.355085,0.387279,0.693713,0.732819
LGBMClassifier(),2.369359,0.043391,0.759095,0.832339,0.372257,0.565359,0.511131,0.768978,0.293391,0.447030,0.697663,0.878875


In [24]:
models_cv_df_style.to_html()

'<style type="text/css">\n#T_dee4d_row0_col0, #T_dee4d_row0_col3, #T_dee4d_row0_col7, #T_dee4d_row1_col5, #T_dee4d_row1_col11, #T_dee4d_row2_col2, #T_dee4d_row2_col6, #T_dee4d_row4_col1, #T_dee4d_row4_col4, #T_dee4d_row4_col10, #T_dee4d_row6_col9, #T_dee4d_row7_col8 {\n  background-color: orange;\n  color: black;\n}\n#T_dee4d_row0_col4, #T_dee4d_row0_col8, #T_dee4d_row0_col10, #T_dee4d_row3_col0, #T_dee4d_row3_col1, #T_dee4d_row4_col3, #T_dee4d_row4_col5, #T_dee4d_row4_col7, #T_dee4d_row4_col9, #T_dee4d_row4_col11, #T_dee4d_row5_col3, #T_dee4d_row5_col5, #T_dee4d_row5_col7, #T_dee4d_row5_col9, #T_dee4d_row5_col11, #T_dee4d_row7_col2, #T_dee4d_row7_col6 {\n  background-color: yellow;\n  color: black;\n}\n</style>\n<table id="T_dee4d_" class="dataframe">\n  <thead>\n    <tr>\n      <th class="blank level0" >&nbsp;</th>\n      <th class="col_heading level0 col0" >mean_fit_time</th>\n      <th class="col_heading level0 col1" >mean_score_time</th>\n      <th class="col_heading level0 col2" 

In [27]:
from IPython.display import HTML

HTML(models_cv_df_style.to_html())

,mean_fit_time,mean_score_time,mean_test_accuracy,mean_train_accuracy,mean_test_f1,mean_train_f1,mean_test_precision,mean_train_precision,mean_test_recall,mean_train_recall,mean_test_roc_auc,mean_train_roc_auc
LogisticRegression(n_jobs=-1),0.739155,0.016379,0.649372,0.653353,0.465099,0.470368,0.370612,0.374926,0.624628,0.631003,0.707766,0.712220
GaussianNB(),0.829836,0.027878,0.681661,0.679474,0.433172,0.429698,0.395317,0.391780,0.500089,0.497647,0.670550,0.671491
KNeighborsClassifier(),0.751049,5.218845,0.582624,0.753072,0.408378,0.638189,0.312137,0.496611,0.590526,0.892708,0.622854,0.898071
SVC(),43.699216,6.156550,0.676836,0.725725,0.434082,0.522247,0.379111,0.454071,0.508247,0.614524,0.678092,0.761598
DecisionTreeClassifier(),1.615918,0.015562,0.652407,1.000000,0.360973,1.000000,0.327325,1.000000,0.402720,1.000000,0.567847,1.000000
RandomForestClassifier(n_jobs=-1),13.703251,0.304821,0.745163,1.000000,0.391103,1.000000,0.470573,1.000000,0.335594,1.000000,0.690340,1.000000
XGBClassifier(n_jobs=-1),6.975440,0.033663,0.743514,0.759182,0.402890,0.439589,0.467566,0.509669,0.355085,0.387279,0.693713,0.732819
LGBMClassifier(),2.369359,0.043391,0.759095,0.832339,0.372257,0.565359,0.511131,0.768978,0.293391,0.447030,0.697663,0.878875
